In [ ]:
import os
import sqlite3
from typing import Annotated, TypedDict

import dotenv
import pandas as pd
import psycopg2
from langchain.tools import tool
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI
from langgraph.graph import END, MessagesState, StateGraph, add_messages
from langgraph.prebuilt import ToolNode

dotenv.load_dotenv()
API_KEY = os.getenv("OPENROUTER_API_KEY")
BASE_URL = os.getenv("API_BASE_URL")
# MODEL_NAME = os.getenv("MODEL_NAME")

DB_CONFIG = {
    "host": "localhost",
    "port": 5432,
    "database": "user_db",
    "user": "user",
    "password": "user",
}

## Поднимаем локальную БД

In [20]:
llm = ChatOpenAI(model="x-ai/grok-4.1-fast:free", base_url=BASE_URL, api_key=API_KEY)

In [ ]:
class AgentState(TypedDict):
    messages: Annotated[list, add_messages]
    raw_data: dict
    aggregated_result: dict

In [ ]:
# @tool
def execute_sql_query(query: str, limit: int = 50) -> dict:
    """
    Выполняет SQL запрос с ограничением на количество строк.

    Args:
        query: SQL запрос (только SELECT)
        limit: Максимум строк (по умолчанию 50)
    """

    # Валидация
    query_upper = query.strip().upper()
    if not query_upper.startswith("SELECT"):
        return {"success": False, "error": "Только SELECT запросы разрешены"}

    if any(keyword in query_upper for keyword in ["DROP", "DELETE", "UPDATE", "INSERT", "ALTER"]):
        return {"success": False, "error": "Опасные операции запрещены"}

    if "LIMIT" not in query_upper:
        query += f" LIMIT {limit}"

    try:
        conn = psycopg2.connect(**DB_CONFIG)
        cursor = conn.cursor()
        cursor.execute(query)

        columns = [desc[0] for desc in cursor.description]
        results = cursor.fetchall()
        row_count = len(results)

        conn.close()

        return {
            "success": True,
            "columns": columns,
            "data": [dict(zip(columns, row)) for row in results],
            "row_count": row_count,
            "truncated": row_count == limit,
        }
    except Exception as e:
        return {"success": False, "error": str(e)}


@tool
def get_table_schema(table_name: str) -> dict:
    """Получает схему таблицы (список колонок и их типы) из базы данных SQLite."""
    conn = sqlite3.connect("database.db")
    cursor = conn.cursor()
    cursor.execute(f"PRAGMA table_info({table_name})")
    columns = cursor.fetchall()
    conn.close()

    return {
        "table": table_name,
        "columns": [{"name": col[1], "type": col[2]} for col in columns],
    }


@tool
def get_districts() -> dict:
    """Возвращает список уникальных районов из таблицы patients."""
    conn = sqlite3.connect("database.db")
    cursor = conn.cursor()
    cursor.execute(
        "SELECT DISTINCT район_проживания FROM patients WHERE район_проживания IS NOT NULL ORDER BY район_проживания"
    )
    districts = [row[0] for row in cursor.fetchall()]
    conn.close()

    return {"districts": districts, "count": len(districts)}

In [42]:
def agent_node(state: MessagesState):
    messages = state["messages"]
    response = llm.bind_tools([sql_query, get_table_schema, get_districts]).invoke(messages)
    return {"messages": [response]}


def should_continue(state: MessagesState):
    messages = state["messages"]
    last_message = messages[-1]

    if hasattr(last_message, "tool_calls") and last_message.tool_calls:
        return "tools"
    return END

In [43]:
workflow = StateGraph(MessagesState)

workflow.add_node("agent", agent_node)
workflow.add_node("tools", ToolNode([sql_query, get_table_schema, get_districts]))

workflow.set_entry_point("agent")

workflow.add_conditional_edges("agent", should_continue, {"tools": "tools", END: END})

workflow.add_edge("tools", "agent")

app = workflow.compile()

In [ ]:
messages = [
    SystemMessage(
        content="""Ты помощник, который может выполнять SQL запросы к базе данных SQLite.

Доступные таблицы: patients, recipes

ВАЖНО: Сначала используй get_table_schema для получения структуры таблицы, затем выполняй SQL запрос.
Используй только синтаксис SQLite."""
    ),
    HumanMessage(
        content="Какая ситуация в красносельском районе? Мне нужен краткий обзор по пациентам и рецептам."
    ),
]

result = app.invoke({"messages": messages})

for m in result["messages"]:
    m.pretty_print()

AI: 
AI: 
AI: В Красносельском районе:

- **Пациенты**: 32 374 всего (17 561 женщин, 14 813 мужчин). Данные о среднем возрасте недоступны.
- **Рецепты**: 0 (нет выписанных рецептов для пациентов района).
